<a href="https://colab.research.google.com/github/VeereshElango/ai_powered_automated_tamil_song_lyrics_generator/blob/master/AutomatedTamilSongGenerationUsingDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutomatedTamilSongGenerationUsingDeepLearning

This notebook was built on top of Tensorflow's tutorial [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation). The data utilized in this notebook can be find [here](https://cutt.ly/tt7krlz).

## Data Import & Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("/content/drive/My Drive/ColabDatasets/Public/tamil_song_lyrics/tamilsonglyrics.csv", index_col=[0])

In [4]:
df= df.sample(frac=1).reset_index(drop=True)
df.head()

,Lyricist,Lyrics
0,NaN,ஆ ஆ ஆ ஹஹஹ்ஹா ஆ ஆ\r\nஇன்பம் ...\r\nஇரவின் அமைதி...
1,NaN,கிளாசுல தூங்க மாட்டோம் வை மச்சி வை\r\nயூ டுபு ...
2,Vaali,பாட்டு தலைவன் பாடினால் பாட்டு தான்\r\nகூட்டம் ...
3,Arivumathi,ஆண் : ஆறடி சுவரு தான் ஆசையை தடுக்குமா\r\nகிளிய...
4,Vairamuthu,ஆண் : குளிச்சா குத்தாலம்\r\nகும்பிட்டா பரமசிவம...


## Data Preparation

In [0]:
# Importing the libraries
import tensorflow as tf
import numpy as np
import os
import time

In [6]:
# Combining all the lyrics to single string variable
text = " ".join([ song for song in df.Lyrics])
print("Total songs combined : {}".format(len(df.Lyrics)))
print ('Length of text: {} characters'.format(len(text)))

Total songs combined : 4142
Length of text: 5266494 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

ஆ ஆ ஆ ஹஹஹ்ஹா ஆ ஆ
இன்பம் ...
இரவின் அமைதியிலே
தென்றல் இனிமையிலே


ஆ .......
இன்பம்
வண்ண நிலவினிலே
இன்பம் பேரின்பம்

இசையே வீணையை மீட்டும்
என் இசையே வீணையை மீட்டும்
சொல்ல எழில் மிகும் வீணை இசைத்தே
ஒரு அன்பு முத்தம் கொடுப்பேன்

இன்பம் ,


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

172 unique characters


## Vectorize the text

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

## Create training examples and targets

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Sample
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

ஆ
 
ஆ
 
ஆ


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'ஆ ஆ ஆ ஹஹஹ்ஹா ஆ ஆ\r\nஇன்பம் ...\r\nஇரவின் அமைதியிலே\r\nதென்றல் இனிமையிலே\r\n\r\n\r\nஆ .......\r\nஇன்பம்\r\nவண்ண நிலவின'
'ிலே\r\nஇன்பம் பேரின்பம்\r\n\r\nஇசையே வீணையை மீட்டும்\r\nஎன் இசையே வீணையை மீட்டும்\r\nசொல்ல எழில் மிகும் வீணை இச'
'ைத்தே\r\nஒரு அன்பு முத்தம் கொடுப்பேன்\r\n\r\nஇன்பம் ,\r\nஇக வாழ்வினிலே\r\nஎனதழியா பேரின்பம்\r\n\r\nஎன் எண்ணம் கைகூட'
'ும்\r\nபொன்னான நாளிதுவே\r\nநீ என்னை பிரியாமல் இருப்பாயா கிளாசுல தூங்க மாட்டோம் வை மச்சி வை\r\nயூ டுபு பீ யூ'
' ஸிக மாட்டோம் வை மச்சி வை\r\nஎஸ் எம் எஸ் பண்ண மாட்டோம் வை மச்சி வை\r\nபிகாஸ் உண்மை எல்லாம் பேச மாட்டோம்\r\n'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(5):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ஆ ஆ ஆ ஹஹஹ்ஹா ஆ ஆ\r\nஇன்பம் ...\r\nஇரவின் அமைதியிலே\r\nதென்றல் இனிமையிலே\r\n\r\n\r\nஆ .......\r\nஇன்பம்\r\nவண்ண நிலவி'
Target data: ' ஆ ஆ ஹஹஹ்ஹா ஆ ஆ\r\nஇன்பம் ...\r\nஇரவின் அமைதியிலே\r\nதென்றல் இனிமையிலே\r\n\r\n\r\nஆ .......\r\nஇன்பம்\r\nவண்ண நிலவின'
Input data:  'ிலே\r\nஇன்பம் பேரின்பம்\r\n\r\nஇசையே வீணையை மீட்டும்\r\nஎன் இசையே வீணையை மீட்டும்\r\nசொல்ல எழில் மிகும் வீணை இ'
Target data: 'லே\r\nஇன்பம் பேரின்பம்\r\n\r\nஇசையே வீணையை மீட்டும்\r\nஎன் இசையே வீணையை மீட்டும்\r\nசொல்ல எழில் மிகும் வீணை இச'
Input data:  'ைத்தே\r\nஒரு அன்பு முத்தம் கொடுப்பேன்\r\n\r\nஇன்பம் ,\r\nஇக வாழ்வினிலே\r\nஎனதழியா பேரின்பம்\r\n\r\nஎன் எண்ணம் கைகூ'
Target data: 'த்தே\r\nஒரு அன்பு முத்தம் கொடுப்பேன்\r\n\r\nஇன்பம் ,\r\nஇக வாழ்வினிலே\r\nஎனதழியா பேரின்பம்\r\n\r\nஎன் எண்ணம் கைகூட'
Input data:  'ும்\r\nபொன்னான நாளிதுவே\r\nநீ என்னை பிரியாமல் இருப்பாயா கிளாசுல தூங்க மாட்டோம் வை மச்சி வை\r\nயூ டுபு பீ ய'
Target data: 'ம்\r\nபொன்னான நாளிதுவே\r\nநீ என்னை பிரியாமல் இருப்பாயா கிளாசுல தூங்க மாட்டோம் வை மச்சி வ

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 2 (' ')
  expected output: 141 ('ஸ')
Step    1
  input: 141 ('ஸ')
  expected output: 144 ('ி')
Step    2
  input: 144 ('ி')
  expected output: 120 ('க')
Step    3
  input: 120 ('க')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 131 ('ம')


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build the model

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 172) # (batch_size, sequence_length, vocab_size)


In [20]:
target_example_batch.shape

TensorShape([64, 100])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           44032     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 172)           176300    
Total params: 13,860,012
Trainable params: 13,860,012
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'een light போல் ஓகே சொல்லும் marriage வாழ்க்கை தானே\r\nநடுவில் ஒரு yellow line தான் novelty\r\nDJ DJ டிஸ்'

Next Char Predictions: 
 "ஞற—ஜ’lSÕகx@0யௌ‘;ºணx.க▪\\ஈ‘$ணDZCRr}`ஃ)‐mஅ௨T`Õஞ– ug<௨3'lMாb}ீºD்N`UனuÕஓb4pHvq\xad`\u200dவM=லNj*OZ▪ा@?gwT¤<▪’ஞ!e"


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 172)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.147275


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss'
    )

In [0]:
EPOCHS=30

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
814/814 [==============================] - 96s 118ms/step - loss: 1.9695
Epoch 2/30
814/814 [==============================] - 96s 117ms/step - loss: 1.5213
Epoch 3/30
814/814 [==============================] - 96s 117ms/step - loss: 1.4123
Epoch 4/30
814/814 [==============================] - 96s 117ms/step - loss: 1.3349
Epoch 5/30
814/814 [==============================] - 95s 117ms/step - loss: 1.2687
Epoch 6/30
814/814 [==============================] - 96s 118ms/step - loss: 1.2092
Epoch 7/30
814/814 [==============================] - 95s 117ms/step - loss: 1.1545
Epoch 8/30
814/814 [==============================] - 96s 117ms/step - loss: 1.1051
Epoch 9/30
814/814 [==============================] - 95s 117ms/step - loss: 1.0600
Epoch 10/30
814/814 [==============================] - 95s 117ms/step - loss: 1.0202
Epoch 11/30
814/814 [==============================] - 95s 117ms/step - loss: 0.9838
Epoch 12/30
814/814 [==============================] - 95s 117ms/step - lo

## Save model

In [29]:
! pip install tensorflowjs

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 92kB 5.1MB/s 
     |████████████████████████████████| 104.6MB 30kB/s 
     |████████████████████████████████| 256kB 56.0MB/s 
     |████████████████████████████████| 921kB 61.9MB/s 
     |████████████████████████████████| 450kB 63.4MB/s 
     |████████████████████████████████| 3.9MB 60.7MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=7dedfe8548bdfbad54c931d6e34d96f8502b3c97f55f775b85d2f172c704ccd6
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=59451d422d617da06f6c089b0619313b47b531aa3aa28edac8048957ba396875
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built PyInquirer gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, bu

In [0]:
import tensorflowjs as tfjs

In [31]:
tfjs.converters.save_keras_model(model, "./drive/My Drive/ColabDatasets/trained_models/fullsongs_model")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [0]:
import json

In [0]:
char2idx_str = json.dumps(char2idx)

In [0]:
with open("./drive/My Drive/ColabDatasets/trained_models/fullsongs_model/char_idx_converter.js","w") as f:
  f.write("var char2idx ="+ str(char2idx)+"; \n var idx2char = "+str(idx2char.tolist()))

## Checking Results

In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            44032     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 172)            176300    
Total params: 13,860,012
Trainable params: 13,860,012
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature=1.0):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 500

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature 
      cat_dist_predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([cat_dist_predicted_id], 0)
      #print(input_eval)
      text_generated.append(idx2char[cat_dist_predicted_id])

  return (start_string + ''.join(text_generated))

In [42]:
print(generate_text(model, start_string=u"அ", temperature=0.8))

அடங்குமா
நீயும் வந்தா கூட
பூ பூக்கவே மறு புறாவே....(துள்ளி)
தாலி கட்ட சேர்ந்த தான்
என்ன கேட்டு கேட்க நீ பார்த்தா
அவக்கை எங்க கூர கொண்டாட்டம்தான்
ஒரு சின்னச் சின்ன அவனைக் கேளு

ஆண்: சுத்தி சுத்தி வந்தீக கோபக் குருவி மலருது Rad
வா வா வா

மண்ணில் இந்தக் குயில் நீயும் வந்தால் ஆடும் இளம் பூங்கொடி!

பறவை பூத்த தோட்டத்தில் ம் பாரு அது
தின்னா தின்னக் தினத்தின்
தின்னா தின்னக்கி திக்கெட் தின்னாரபரு
எல்லாம் பொய்யும் புரியலையே

பாட்டு ஒன்னு கட்டி வச்சேனே வா வா வா
கட்டிக்கிட நான் தொட்டதெல்ல
